In [2]:
qpath = r'../../' # path to quetzal here
data = r'inputs/'
import sys
sys.path.insert(0, qpath)

# import class
from quetzal.io.gtfs_reader import importer

SQLalchemy is not installed. No support for SQL output.


In [3]:
import numpy as np

### Read GTFS

In [4]:
feed = importer.GtfsImporter(path=data + r'bilbao.zip', dist_units='m')
feed = feed.clean()
feed.describe()

,indicator,value
0,agencies,[Bilbobus]
1,running_services,[1]
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20200310
5,num_routes,82
6,num_trips,3297
7,num_stops,498
8,num_shapes,0
9,num_frequencies,0


In [5]:
feed.validate()

,type,message,table,rows
0,warning,"Repeated pair (trip_id, departure_time)",stop_times,"[128, 4087, 4101, 4115, 4129, 4143, 4157, 4171..."


In [6]:
feed.map_stops(feed.stops.stop_id)

In [7]:
feed = feed.create_shapes()

In [8]:
feed.map_trips(
    feed.trips.groupby('route_id').first().trip_id.head(50)
)

Frequency conversion currently work only for one specific service and date, and on one given time period.  
It computes the average headway over this time period.

### Restrict to one date and merge services

In [9]:
feed = feed.restrict(dates=['20191210'])
feed.group_services()

In [10]:
feed.describe()

,indicator,value
0,agencies,[Bilbobus]
1,running_services,[1]
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20191210
5,num_routes,82
6,num_trips,3297
7,num_stops,498
8,num_shapes,124
9,num_frequencies,0


### Build simplified patterns by clustering stops

In [11]:
feed.build_stop_clusters(distance_threshold=300)

In [12]:
print('N stops:', len(feed.stops))
print('N clusters:', len(feed.stops.cluster_id.unique()))

N stops: 498
N clusters: 165


In [13]:
feed.build_patterns(on='cluster_id')

In [14]:
print('N trips:', len(feed.trips))
print('N patterns:', len(feed.trips.pattern_id.unique()))
print('N routes:', len(feed.trips.route_id.unique()))

N trips: 3297
N patterns: 124
N routes: 82


In [15]:
feed.describe()

,indicator,value
0,agencies,[Bilbobus]
1,running_services,[1]
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20191210
5,num_routes,82
6,num_trips,3297
7,num_stops,498
8,num_shapes,124
9,num_frequencies,0


### Convert to frequencies

In [16]:
feed.stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,406305,06:15:00,06:15:00,5405,1,NaN,0,0,NaN
1,406305,06:17:00,06:17:00,5106,2,NaN,0,0,NaN
2,406305,06:21:00,06:21:00,5110,3,NaN,0,0,NaN
3,406305,06:23:00,06:23:00,2108,4,NaN,0,0,NaN
4,406305,06:24:00,06:24:00,2104,5,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...
57134,436376,14:03:00,14:03:00,5101,8,NaN,0,0,NaN
57135,436376,14:08:00,14:08:00,5704,9,NaN,0,0,NaN
57136,436376,14:11:00,14:11:00,5601,10,NaN,0,0,NaN
57137,436376,14:13:00,14:13:00,5701,11,NaN,0,0,NaN


In [15]:
time_range = ['08:00:00', '10:00:00']  # time format must be HH:MM:SS
feed_f = feed.convert_to_frequencies(time_range=time_range)

100%|██████████████████████████████████████████████████████████████████████████████████| 75/75 [00:00<00:00, 79.03it/s]


In [1]:
feed_f = feed.convert_to_frequencies(time_range=time_range)

NameError: name 'feed' is not defined

In [16]:
feed_f.describe()

,indicator,value
0,agencies,[Bilbobus]
1,running_services,[1]
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20191210
5,num_routes,67
6,num_trips,75
7,num_stops,495
8,num_shapes,75
9,num_frequencies,75


### Build nodes and links

In [17]:
feed_f.build_links_and_nodes()

In [18]:
feed_f.nodes.head(1).T

,0
stop_id,2201
stop_code,NaN
stop_name,Anselma de Salces (2)
stop_desc,NaN
zone_id,NaN
stop_url,NaN
location_type,NaN
cluster_id,85
geometry,POINT (506258.4033893289 14790384.01825237)


In [19]:
feed_f.links.head(1).T

,0
index,0
a,1604
trip_id,189_0
link_sequence,1
departure_time,27900
b,1605
arrival_time,27960
time,60
headway,900
pattern_id,189_0


## All in one
While we recommand to build the nodes and links gradually by exploring the GTFS data first, it is also possible to do it in one line.

In [20]:
feed = importer.GtfsImporter(path=data + r'bilbao.zip', dist_units='m')
feed.describe()

,indicator,value
0,agencies,[Bilbobus]
1,running_services,[1]
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20200310
5,num_routes,92
6,num_trips,3297
7,num_stops,498
8,num_shapes,0
9,num_frequencies,0


In [21]:
imp = feed.build(
    date='20191210',
    time_range=['08:00:00', '10:00:00'],
    cluster_distance_threshold=300 # by default: None and no clustering
)

Restricting to date…
Grouping services…
Cleaning…
Clustering stops…
Building patterns…


  8%|██████▋                                                                            | 6/75 [00:00<00:01, 58.81it/s]

Converting to frequencies…


100%|██████████████████████████████████████████████████████████████████████████████████| 75/75 [00:00<00:00, 75.68it/s]


Building links and nodes…


In [22]:
imp.describe()

,indicator,value
0,agencies,[Bilbobus]
1,running_services,[1]
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20191210
5,num_routes,67
6,num_trips,75
7,num_stops,495
8,num_shapes,0
9,num_frequencies,75


In [23]:
imp.links.head()

,index,a,trip_id,link_sequence,departure_time,b,arrival_time,time,headway,pattern_id,...,direction_id,shape_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,geometry
0,0,1604,189_0,1,27900.0,1605,27960.0,60.0,900.0,189_0,...,NaN,NaN,27,01_IDA,Arangoiti - Plaza Biribila,NaN,3,NaN,NaN,"LINESTRING (504266.448 14791400.960, 504088.19..."
1,1,1605,189_0,2,27960.0,1702,28140.0,180.0,900.0,189_0,...,NaN,NaN,27,01_IDA,Arangoiti - Plaza Biribila,NaN,3,NaN,NaN,"LINESTRING (504088.197 14791399.303, 503796.02..."
2,2,1702,189_0,3,28140.0,1703,28200.0,60.0,900.0,189_0,...,NaN,NaN,27,01_IDA,Arangoiti - Plaza Biribila,NaN,3,NaN,NaN,"LINESTRING (503796.023 14791563.341, 503745.17..."
3,3,1703,189_0,4,28200.0,1306,28440.0,240.0,900.0,189_0,...,NaN,NaN,27,01_IDA,Arangoiti - Plaza Biribila,NaN,3,NaN,NaN,"LINESTRING (503745.174 14791526.405, 503618.05..."
4,4,1306,189_0,5,28440.0,1414,28680.0,240.0,900.0,189_0,...,NaN,NaN,27,01_IDA,Arangoiti - Plaza Biribila,NaN,3,NaN,NaN,"LINESTRING (503618.056 14791226.062, 504049.58..."


In [24]:
imp.nodes.head()

,stop_id,stop_code,stop_name,stop_desc,zone_id,stop_url,location_type,cluster_id,geometry
0,2201,NaN,Anselma de Salces (2),NaN,NaN,NaN,NaN,85,POINT (506258.403 14790384.018)
1,2417,NaN,Trauko (14),NaN,NaN,NaN,NaN,10,POINT (506528.037 14790435.054)
2,3103,NaN,"Kepa Enbeitia ""Urretxindorra""",NaN,NaN,NaN,NaN,157,POINT (508092.006 14789626.524)
3,6209,NaN,Gregorio la Revilla 1,NaN,NaN,NaN,NaN,56,POINT (504935.494 14790091.282)
4,6210,NaN,Gregorio la Revilla 17,NaN,NaN,NaN,NaN,79,POINT (504867.070 14789866.493)
